###Importing libraries

In [0]:
import requests
import pandas as pd
from io import StringIO

###Setting parameters

In [0]:
symbol = "MSFT"
apikey = "3VABX86SUL1NE9NX"
datatype = "csv"

###API request

In [0]:
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={apikey}&datatype={datatype}"
r = requests.get(url)
data = pd.read_csv(StringIO(r.text))
print(data.head())

###Transforming the data

In [0]:
# Rename columns (already fine, but standardizing)
data = data.rename(columns={
    'timestamp': 'date',
})

# Convert date to datetime type
data['date'] = pd.to_datetime(data['date'])

# Ensure numeric columns are correct types
numeric_cols = ['open', 'high', 'low', 'close', 'volume']
data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric)

# Sort by date ascending
data = data.sort_values('date')

# Preview transformed data
print(data.head())


In [0]:
# Find the latest date in the dataset
latest_date = data['date'].max()
print("Latest date in dataset:", latest_date)


###Loading to Spark Dataframe

In [0]:
%sql
use catalog personal_catalog

In [0]:
%sql
create schema MSFT_stock_prices

In [0]:
%sql
use schema MSFT_stock_prices

In [0]:
spark_df = spark.createDataFrame(data)
spark_df.write.mode("overwrite").saveAsTable("personal_catalog.MSFT_stock_prices.MSFTstock_prices")
print(spark_df.head(5))


In [0]:
df = spark.read.table("personal_catalog.MSFT_stock_prices.MSFTstock_prices")
display(df.limit(5))

In [0]:
import matplotlib.pyplot as plt
import pandas as pd

# Calculate 6-month rolling average of closing price
data['rolling_avg_6m'] = data['close'].rolling(window=6).mean()

# Plot original close price and rolling average
plt.figure(figsize=(12,6))
plt.plot(data['date'], data['close'], marker='o', linestyle='-', color='blue', label='Close Price')
plt.plot(data['date'], data['rolling_avg_6m'], linestyle='--', color='red', label='6-Month Rolling Avg')
plt.title(f"{symbol} Monthly Closing Price with 6-Month Rolling Average")
plt.xlabel("Date")
plt.ylabel("Price")
plt.grid(True)
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [0]:
import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Filter historical data from 2012
historical_data = data[data['date'] >= '2012-01-01'].copy()
ts = historical_data.set_index('date')['close']

# Fit Holt-Winters model with additive trend (no seasonality)
model = ExponentialSmoothing(ts, trend='add', seasonal=None)
fit = model.fit()

# Forecast next 12 months
forecast_12m = fit.forecast(12)
forecast_dates = pd.date_range(historical_data['date'].max() + pd.offsets.MonthEnd(1), periods=12, freq='M')

# Create interactive plot
fig = go.Figure()

# Historical trace
fig.add_trace(go.Scatter(
    x=historical_data['date'],
    y=historical_data['close'],
    mode='lines+markers',
    name='Historical Close',
    line=dict(color='blue'),
    marker=dict(size=6)
))

# Forecast trace
fig.add_trace(go.Scatter(
    x=forecast_dates,
    y=forecast_12m,
    mode='lines+markers',
    name='12-Month Forecast',
    line=dict(color='red', dash='dash'),
    marker=dict(size=6)
))

# Layout
fig.update_layout(
    title=f"{symbol} Historical Close (2012-Present) + 12-Month Forecast",
    xaxis_title="Date",
    yaxis_title="Close Price",
    hovermode="x unified"
)

fig.show()


